# Imports

In [53]:
import sys
import os
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt

# Constants

In [54]:
print(os.getcwd())

/home/berimbolo/AdvancedComputerNetworks/CSC845-twitter-spam-detection/alex


In [55]:
TRAIN_DIR = 'datasets/train'
TRAIN_PATH = 'datasets/train/train.csv'
EXAMPLE_FILE = 'datasets/SpamData/01_Processing/practice_email.txt'

# Reading Test Data

In [56]:
data = pd.read_csv(TEST_DATA)
tweets = DataFrame(data, columns=['Tweet'])
followings = DataFrame(data, columns=['following'])

# Plotting Data

In [57]:
data['Tweet'].iloc[0]

'Good Morning Love  @LeeBrown_V'

# Tweet Body Extraction

In [84]:
#stream = open('datasets/train.csv')

def tweet_body_generator(path):
    filepath = join(root, file_name)
    stream = open(filepath)
    lines = []

    for line in stream:
        lines.append(line)

    tweet_body = '\n'.join(lines)
    yield file_name, tweet_body
                
    stream.close()


In [89]:

def df_from_file(path):
    rows = []
    row_names = []
    
    for tweet_body, classification in tweet_body_generator(path):
        rows.append({'MESSAGE': tweet_body, 'CATEGORY':classification})
        row_names.append(file_name)
        
    return pd.DataFrame(rows)

In [90]:
spam_tweets = df_from_directory(TRAIN_PATH, 1)

NameError: name 'tweet_body' is not defined